# Melify
At the moment, because it is rather complecated, we shall first take the test set and get the mean 
and variance. In this way we can proceed without conducting any preprocessing. The only change is that we will fix the normalization here and pass in the data for all trainings and testings. 

However, because at the moment I haven't found a good way to preserve good variance if using online methods, I will use the mean and variance as a whole, but from test set here to get all. 

In [1]:
from model_dataset import TargetVowelDatasetManualNorm
from model_dataset import MelSpecTransformDBNoNorm as TheTransform
from paths import *
from model_dataset import Normalizer, DeNormalizer, TokenMap

import pandas as pd
import pickle

In [2]:
REC_SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 64
LOADER_WORKER = 32

In [13]:
t_set = pd.read_csv(os.path.join(src_, "phi-T-guide.csv"))
st_set = pd.read_csv(os.path.join(src_, "phi-ST-guide.csv"))

t_set_sampled = t_set.sample(n=len(st_set))

integrated = pd.concat([t_set_sampled, st_set], ignore_index=True, sort=False)
integrated = integrated.sample(frac=1).reset_index(drop=True)

In [15]:
mytrans = TheTransform(sample_rate=REC_SAMPLE_RATE, 
                    n_fft=N_FFT, n_mels=N_MELS)

with open(os.path.join(src_, "no-stress-seg.dict"), "rb") as file:
    # Load the object from the file
    mylist = pickle.load(file)
    mylist = ["BLANK"] + mylist
    mylist = mylist + ["SIL"]

# Now you can use the loaded object
mymap = TokenMap(mylist)

mynorm = Normalizer(Normalizer.norm_mvn_manual)

In [16]:
ds = TargetVowelDatasetManualNorm(
    src_dir=train_cut_phone_, guide_=integrated, 
    mapper=mymap, transform=mytrans, normalizer=mynorm, 
    noise_fixlength=False, noise_amplitude_scale=0.004, mv_config=None
)

HarmonicNoiseGen: sample_rate=16000, amplitude_scale=0.004, oscillation_scale=0.0006, f_0=50
No mean and variance provided, calculating from the data ...


In [17]:
ds.mean.item(), ds.std.item()

(-22.27385902404785, 16.151123046875)

In [18]:
mean_std_dict = {'mean': ds.mean.item(), 'std': ds.std.item()}

with open(os.path.join(src_, "mv_config.pkl"), "wb") as file:
    pickle.dump(mean_std_dict, file)

Now we have saved the mean and variance, we can use them to normalize the data. 